# Script para extraer información del Directorio de Proveedores y Contratistas Sancionados.

Directorio de Licitantes, Proveedores y Contratistas sancionados con el impedimento para presentar propuestas o celebrar contratos con las dependencias, entidades de la Administración Pública Federal y de los Gobiernos de los Estados.

Ultima fecha de actualización:03/Marzo/2025

In [1]:
import requests
import json
import pandas as pd
from IPython.display import display

url = 'https://servicios-estrategico.funcionpublica.gob.mx/particularesSancionadosPro'

# Modificamos el payload eliminando el filtro de numero_expediente y rfc para extraer todosblos expedientes,
#si se desea un expediente en particular se agrega siguiendo la sintaxis ---->  filtros: {numero_expediente: "expediente que se busca", rfc: "rfc que se busca"}
payload = {
    "operationName": "getExpediente",
    "variables": {
        "filtros": {}  # Aquí vaciamos el filtro para que devuelva todos los expedientes
    },
    "query": """
    query getExpediente($filtros: FiltrosInput!) {
        results(filtros: $filtros) {
            numero_expediente
            numero_procedimiento_adm
            numexpediente_corto
            numprocedimiento_corto
            rfc
            nombre_razon_social
            institucion_dependencia {
                nombre
            }
            fecha_notificacion
            multa {
                monto
                monto_cf
            }
            plazo {
                fecha_inicial
                fecha_final
                plazo_inha
            }
            observaciones
            objeto_social
            leyes_infringidas
            causa_motivo_hechos
            autoridad_sancionadora
            fecha_dof
            responsable {
                nombres_resp
                primer_apellido_resp
                segundo_apellido_resp
                cargo_resp
                telefono_resp
                email_resp
            }
            telefono
        }
    }
    """
}

response = requests.post(url, json=payload)
data = response.json()

# Imprimir los resultados en formato JSON
#with open('respuesta_expedientes.json', 'w') as f:
  #  json.dump(data, f, indent=4)


Se desea obtener la información en formato estructurado

In [2]:
# Extraer los resultados de la respuesta
results = data.get('data', {}).get('results', [])

# Si la clave 'results' existe, la convertimos a un DataFrame
if results:
    # Crear un DataFrame a partir de la lista de resultados
    df = pd.json_normalize(results, sep='_')

    # Mostrar el DataFrame
    display(df.head(5))  # Mostrar las primeras filas para ver cómo luce
else:
    print("No se encontraron expedientes.")

,numero_expediente,numero_procedimiento_adm,numexpediente_corto,numprocedimiento_corto,rfc,nombre_razon_social,fecha_notificacion,observaciones,objeto_social,leyes_infringidas,...,multa_monto_cf,plazo_fecha_inicial,plazo_fecha_final,plazo_plazo_inha,responsable_nombres_resp,responsable_primer_apellido_resp,responsable_segundo_apellido_resp,responsable_cargo_resp,responsable_telefono_resp,responsable_email_resp
0,090850081/2005,09085PA-0081/2005,0081/2005,PA-0081/2005,DMA860923MH1,"DEMOLICIONES Y MATERIALES, S.A. DE C.V.",2007-02-02,"NO HA REALIZADO EL PAGO DEL CREDITO FISCAL, PO...",None,LEY DE OBRAS PÚBLICAS Y SERVICIOS RELACIONADOS...,...,"67,860.00",2007-02-15,2007-05-15,3 meses,Rogelio Arturo,Aviña,Martínez,TITULAR DEL ÓRGANO INTERNO DE CONTROL,5551331000 ext 1025,avinam@asa.gob.mx
1,090850001/2024,09085PA-0007/2024,0001/2024,PA-0007/2024,LOLD860221ND2,DANIEL GASPAR LOPEZ LANZ,2024-12-17,"LA SANCIÓN SE ENCUENTRA SUBSISTENTE, NO HA INT...",None,"LEY DE ADQUISICIONES, ARRENDAMIENTOS Y SERVICI...",...,"912,240.00",2025-01-25,2026-10-25,"1 año, 9 meses",Rogelio Arturo,Aviña,Martínez,TITULAR DEL ÓRGANO INTERNO DE CONTROL,5551331000 ext 1025,avinam@asa.gob.mx
2,000070021/2022,00007PA-0021/2022,0021/2022,PA-0021/2022,CPU150813UE7,COMERCIALIZADORA PUAN S.A. DE C.V.,2024-07-16,SE INHABILITA POR EL TERMINO DE 7 MESES Y UNA ...,None,"LEY DE ADQUISICIONES, ARRENDAMIENTOS Y SERVICI...",...,"625,959.75",2023-12-01,2024-07-01,7 meses,CARLOS ARTURO,PANCARDO,ESCUDERO,TITULAR DEL ÓRGANO INTERNO DE CONTROL,None,inspcont_ar_resp@sedena.gob.mx
3,006410117/2023,00641PA-0045/2023,0117/2023,PA-0045/2023,SSS160921JC6,"SYS SUMINISTROS Y SOLUCIONES, S.A. DE C.V.",2025-02-13,YA SE NOTIFICO LA RESOLUCION,None,"LEY DE ADQUISICIONES, ARRENDAMIENTOS Y SERVICI...",...,"27,782.20",None,None,None,ALBERTO,MOCIÑOZ,GUTIERREZ,TITULAR DEL AREA DE RESPONSABILIDADES,None,alberto.mocinoz@imss.gob.mx
4,091200005/2024,09120PA-0013/2024,0005/2024,PA-0013/2024,ZDE120713BN5,"ZITUM DESARROLLADORES, S.A. DE C.V.",2025-01-31,CON FECHA 14 DE FEBRERO DE 2025 SE PUBLICO EN ...,None,LEY DE OBRAS PÚBLICAS Y SERVICIOS RELACIONADOS...,...,"157,685.00",2025-02-15,2025-05-15,3 meses,FABIOLA,BELLO,DURAN,TITULAR DEL AREA DE RESPONSABILIDADES,None,fbellod@capufe.gob.mx


Se guarda en un archivo excel

In [3]:
df.to_excel('prov_sancionados_contratistas.xlsx', index=False, engine='openpyxl')